In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pickle

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [2]:
def onehot(vals, possible_vals):
    if not isinstance(possible_vals, list): raise TypeError("provide possible_vals as a list")
    enc_vals = np.zeros([len(vals), len(possible_vals)])
    for i, value in enumerate(vals):
        if isinstance(possible_vals[0], float):
            enc = np.where(abs(possible_vals-value)<1e-3)
        else:
            enc = np.where(possible_vals==value)
        enc_vals[i,enc] = 1
    return enc_vals

In [3]:
class Rescale(object):
    def __init__(self, scalar):
        self.scalar = scalar

    def __call__(self, im):
        w, h = [int(s*self.scalar) for s in im.size]
        return transforms.Resize((h, w))(im)

class Crop(object):
    def __init__(self, box):
        assert len(box) == 4
        self.box = box

    def __call__(self, im):
        return im.crop(self.box)

class Augment(object):
    def __init__(self, seq):
        self.seq = seq

    def __call__(self, im):
        return Image.fromarray(self.seq.augment_images([np.array(im)])[0])

def get_data_transforms(t='train'):
    data_transforms = {
        'train': transforms.Compose([Crop((0,120,800,480)),
            Rescale(0.4),
            transforms.ToTensor()
        ]),
        'val': transforms.Compose([
            Crop((0,120,800,480)),
            Rescale(0.4),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms[t]

In [4]:
from torch.utils.data import Dataset
from matplotlib import image
from matplotlib import pyplot
from torchvision import transforms
from PIL import Image

#LABEL_KEYS = ['v_throttle', 'v_break', 'v_steer', 'traffic_light', 'front_vehicle', 'centre_dist', 'pedestrian_distance']

class CARLA_Dataset(Dataset):
    def __init__(self, t, pickle_file_path_l, image_dir_path, pickle_file_path_r):
        #self.inputs, self.labels = {}, {}
        #initializing transforms
        assert t in ['train', 'val']
        self.transform = get_data_transforms(t)
        self.labels = {}    
        ###############LEFT CAMERA##############
        #reading the title of all images to access the pickle file
        self.image_path = image_dir_path
        self.image_list = os.listdir(image_dir_path)
        #self.image_list_l.remove('.DS_Store')
        self.file_name = []
        for file in self.image_list:
            self.file_name.append(os.path.splitext(file)[0])
            
        #initialize the labels 

        
        #read pickle file
        self.pickle_list_l = os.listdir(pickle_file_path_l)
        self.pickled_data_l = {}
        for file in self.pickle_list_l:
            f = open((pickle_file_path_l + file), 'rb')  
            self.pickled_data_l.update(pickle.load(f))
            f.close() 
        
        ###############RIGHT CAMERA##############
    
        
        #read pickle file
        self.pickle_list_r = os.listdir(pickle_file_path_r)
        self.pickled_data_r = {}
        for file in self.pickle_list_r:
            f = open((pickle_file_path_r + file), 'rb')  
            self.pickled_data_r.update(pickle.load(f))
            f.close() 
        
        # transform reg affordances
        #reg_keys = ['front_vehicle', 'centre_dist', 'pedestrian_distance']
        #for key in self.pickled_data:
        #   entry = self.pickled_data[key]
            #print("1.", entry['front_vehicle'])
        #reg_norm = entry[reg_keys] / abs(entry[reg_keys]).max()
                # transform cls affordances
      
        f.close() 

    def __len__(self):
        return len(self.file_name)

    def __getitem__(self, idx):
        frames      = []
        inputs = {}
            
        #####################LEFT CAMERA################
        #reading PIL
        self.raw_image = Image.open(os.path.join(self.image_path, self.image_list[idx])).convert('RGB')
        #pyplot.imshow(self.raw_image_l)
        #pyplot.show()
        
        #reading file name to access the pickle file
        current_fname = self.file_name[idx]
        chk_camera = current_fname[-3:]

        if chk_camera == '_rt' :
            current_fname_r = current_fname[:-3]
            label_dict = self.pickled_data_r[current_fname_r] 

        else :

            label_dict = self.pickled_data_l[current_fname] 
            
        label_values = list(label_dict.values())
        
        #transforming regression labels
        self.labels['front_vehicle'] = torch.Tensor(np.array(float(label_dict['front_vehicle'])))
        self.labels['centre_dist'] = torch.Tensor(np.array(float(label_dict['centre_dist'])))
        self.labels['pedestrian_distance'] = torch.Tensor(np.array(float(label_dict['pedestrian_distance'])))
        
        #transforming classification labels
        #self.labels_l['traffic_light'] = torch.Tensor(onehot(np.array(label_dict['traffic_light']), [False, 'Green', True, 'Red']))
        #transforming raw PIL image
        im = self.transform(self.raw_image)
        
        inputs ['sequence'] = im
        
        #print(inputs ['sequence'].dim())
        return inputs, self.labels

            
        ###############RIGHT CAMERA####################
        #reading PIL
        #self.raw_image_r = Image.open(os.path.join(self.image_path_r, self.image_list_r[idx])).convert('RGB')
        #pyplot.imshow(self.raw_image_r)
        #pyplot.show()
        
        #reading file name to access the pickle file
        #current_fname_r = self.file_name_r[idx]
        #label_dict_r = self.pickled_data_r[current_fname_r]       
        #label_values_r = list(label_dict_r.values())
       
        #transforming regression labels
        #self.labels_r['front_vehicle'] = torch.Tensor(np.array(float(label_dict_r['front_vehicle'])))
        #self.labels_r['centre_dist'] = torch.Tensor(np.array(float(label_dict_r['centre_dist'])))
        #self.labels_r['pedestrian_distance'] = torch.Tensor(np.array(float(label_dict_r['pedestrian_distance'])))
        
        #transforming classification labels
       # self.labels_r['traffic_light'] = torch.Tensor(onehot(np.array(label_dict_r['traffic_light']), [False, 'Green', True, 'Red']))
        
        #transforming raw PIL image
       # im_r = self.transform(self.raw_image_r)
        
        #inputs ['sequence'] = im_r
            
        #return transformed image and labels
        #sample = {'image_l' : im_l, 'labels_l' : self.labels_l ,'image_r' : im_r, 'labels_r' : self.labels_r}
        #return sample

        
        #self.labels['v_break'] = torch.Tensor(onehot(np.array(list(label_dict['v_break'])), [0.0, 1.0]))
        #print(label_dict['traffic_light'])
        #print(self.labels['traffic_light'])
        #print(label_dict['v_break'])
        #print(self.labels['v_break'])
        #pyplot.imshow(self.raw_image)
        #pyplot.show()
        #reg_keys = ('front_vehicle', 'centre_dist', 'pedestrian_distance', 'v_steer')
        #reg_norm = label_dict[reg_keys] / abs(label_dict[reg_keys]).max()
        #reg_norm = label_dict[reg_keys]

        #for i in 'front_vehicle', 'centre_dist', 'pedestrian_distance', 'v_steer':
        #  reg_norm = label_dict[i] / abs(label_dict[i])
        #  print(i, reg_norm)
        #print(type(reg_keys))
        #reg_norm = label_dict['front_vehicle', 'centre_dist'] / max(abs(label_dict['front_vehicle', 'centre_dist']))

In [5]:
#C_dataset = CARLA_Dataset( 'train', pickle_file_path_l='/Users/seimasaki/CE7454_2019/codes/data/Left_pickle_file/', image_dir_path='/Users/seimasaki/CE7454_2019/codes/data/Left_camera/', pickle_file_path_r='/Users/seimasaki/CE7454_2019/codes/data/Right_pickle_file/')

#for i in range(len(C_dataset)):
  #  sample = C_dataset[i]
   # print(sample[0])
 #print(sample['labels_l'])

In [6]:
def get_data(batch_size):
    train_ds = CARLA_Dataset( 'train', pickle_file_path_l='/Users/seimasaki/CE7454_2019/codes/data/Left_pickle_file/',
                                    image_dir_path='/Users/seimasaki/CE7454_2019/codes/data/Camera/', 
                                    pickle_file_path_r='/Users/seimasaki/CE7454_2019/codes/data/Right_pickle_file/')
    val_ds = CARLA_Dataset( 'val', pickle_file_path_l='/Users/seimasaki/CE7454_2019/codes/data/Left_pickle_file/',
                                    image_dir_path='/Users/seimasaki/CE7454_2019/codes/data/Camera/', 
                                    pickle_file_path_r='/Users/seimasaki/CE7454_2019/codes/data/Right_pickle_file/')
    return (len(train_ds),len(val_ds),DataLoader(train_ds, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=10),
        DataLoader(val_ds, batch_size=batch_size*2, pin_memory=True, num_workers=10),
    )

In [7]:
#print(get_data(100))

In [8]:

##########################################################
# pytorch util functions 
##########################################################

def load_checkpoint(filename) :
    #print(filename)
    checkpoint = torch.load(filename)
    model_state_dict = checkpoint['model_state_dict']
    optimizer_state_dict = checkpoint['optimizer_state_dict']
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return [model_state_dict, optimizer_state_dict, epoch, loss]

def save_checkpoint(model, optimizer, epoch, loss, filename):
    print("Saving Checkpoint....")
    model_state_dict = model.state_dict()
    optimizer_state_dict = optimizer.state_dict()
#     print("model_state_dict : ", model_state_dict)
#     print("optimizer_state_dict : ", optimizer_state_dict)
#     print("Epoch : ", epoch)
#     print("filename : ", filename)

    torch.save({
            'epoch': epoch,
            'model_state_dict': model_state_dict,
            'optimizer_state_dict': optimizer_state_dict,
            'loss': loss
            }, filename)
    print("Saving Checkpoint Done")

def display_image(image) : 
    plt.show(image)

In [9]:
import numpy as np
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch import optim
import torch.nn as nn
#from ipdb import set_trace
from datetime import datetime
import time
import copy
from util_funcs import *
from tqdm import tqdm
from ipdb import set_trace
import argparse

In [10]:
import sys
#sys.path.insert(0, "../prev_work/CAL/training")

#from train import fit, custom_loss, validate
from metrics import calc_metrics

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
def convert_inputs(inputs_old, labels):
    inputs            = inputs_old['sequence']
    #print("labels keys : ", labels.keys())
    #labels = list(sample['labels_l'].values())
    labels_vdistance  = labels['front_vehicle']
    labels_cdistance  = labels['centre_dist']
    labels_direction  = labels['pedestrian_distance']
    combined_label    = torch.stack([labels_vdistance, labels_cdistance, labels_direction], dim=1)
    #print("combined_label shape : ", combined_label.size())
    return inputs, combined_label
    #return combined_label

In [13]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, num_epochs=30):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 0.0
    epoch_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            #for inputs, labels in dataloaders[phase]:
            for i, data in enumerate(dataloaders[phase]) :
            #for i, data in tqdm(dataloaders[phase]) :
                inputs_old = data[0]
                #print("input before convertion", inputs_old)
                labels = data[1]
                inputs, labels = convert_inputs(inputs_old ,labels)
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #_, preds = torch.max(outputs, 1)
                    preds = outputs
                    print(outputs.size(), labels.size())
                    #input("Enter")
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        if(i%100 == 0) :
                            print("preds shape : ", preds.size(), " labels shape : ", labels.size())
                            print("preds : ", preds[0].T.data, "\nlabels : ", labels.data[0].T.data)
                            print("loss :", loss.item())

                # statistics
                running_loss += loss.item() * inputs.size(0)
                #running_corrects += torch.sum(preds == labels.data)

                if(phase == 'val') :
                    if(i%10==0) :
                        print("preds : ", preds[0].T.data, "\nlabels : ", labels.data[0].T.data)
                        print("loss :", loss.item())
                #if(i==10) : 
                #    break
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            #epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            #if phase == 'val' and epoch_acc > best_acc:
            if phase == 'val' and epoch_loss < best_loss:
                #best_acc = epoch_acc
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'train'  :
                checkpoint_name = 'checkpoints/model_' + str(epoch) + '.tar'
                save_checkpoint(model, optimizer, epoch, epoch_loss, checkpoint_name)
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [14]:
def load_checkpoint_weights(checkpoint, model) :
    print("checkpoint name : ", checkpoint)
    _model_state_dict, _optimizer_state_dict, _epoch, _loss = load_checkpoint(checkpoint)
    model.load_state_dict(_model_state_dict)

def resume_model(checkpoint, model, optimizer) : 
    print("checkpoint name : ", checkpoint)
    _model_state_dict, _optimizer_state_dict, _epoch, _loss = load_checkpoint(checkpoint)
    #print(_model_state_dict)
    model.load_state_dict(_model_state_dict)
    optimizer.load_state_dict(_optimizer_state_dict)
    print("resume model succesful")

In [15]:
len_train_ds, len_valid_ds, train_dl, valid_dl = get_data(batch_size=128)
print(len_train_ds)
print(len_valid_ds)

dataloaders   = {'train':train_dl, 'val':valid_dl}
print(dataloaders['train'])
dataset_sizes = {'train':len_train_ds, 'val':len_valid_ds}

48
48


In [16]:
print("dataset sizes :", dataset_sizes)

dataset sizes : {'train': 48, 'val': 48}


In [17]:
#model definitions
model_ft         = models.resnet50(pretrained=True)
criterion        = nn.MSELoss()
optimizer_ft     = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
#load_checkpoint_weights(args.checkpoint, model_ft)
num_ftrs         = model_ft.fc.in_features
model_ft.fc      = nn.Linear(num_ftrs, 3)
model_ft         = nn.DataParallel(model_ft)
model_ft         = model_ft.to(device)
print("copying model to device....", device)

copying model to device.... cpu


In [18]:
#train model
print("starting training...")
#if(args.retrain == True) : 
 #   resume_model(args.checkpoint, model_ft, optimizer_ft)
train_model(model_ft, dataloaders, dataset_sizes, criterion, optimizer_ft, exp_lr_scheduler, 2)

starting training...
Epoch 0/1
----------
torch.Size([48, 3]) torch.Size([48, 3])
preds shape :  torch.Size([48, 3])  labels shape :  torch.Size([48, 3])
preds :  tensor([ 0.1922, -0.1138, -0.3542]) 
labels :  tensor([50.0000,  0.4116, 50.0000])
loss : 1679.7928466796875
train Loss: 1679.7928 Acc: 0.0000
Saving Checkpoint....
Saving Checkpoint Done
torch.Size([48, 3]) torch.Size([48, 3])
preds :  tensor([ 0.2485,  0.2698, -0.5820]) 
labels :  tensor([50.0000,  0.6656, 50.0000])
loss : 1681.686767578125
val Loss: 1681.6868 Acc: 0.0000

Epoch 1/1
----------
torch.Size([48, 3]) torch.Size([48, 3])
preds shape :  torch.Size([48, 3])  labels shape :  torch.Size([48, 3])
preds :  tensor([ 0.2457, -0.2158, -0.4988]) 
labels :  tensor([50.0000,  0.4116, 50.0000])
loss : 1672.541259765625
train Loss: 1672.5413 Acc: 0.0000
Saving Checkpoint....
Saving Checkpoint Done
torch.Size([48, 3]) torch.Size([48, 3])
preds :  tensor([ 0.2559,  0.2406, -0.5408]) 
labels :  tensor([50.0000,  0.6656, 50.0000]

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      